In [26]:
# import necessary packages
import os
import matplotlib.pyplot as plt
import geopandas as gpd
from descartes import PolygonPatch
import pandas as pd
import numpy as np
import seaborn as sns
import fiona

In [43]:
buurten = gpd.read_file("data/neighborhoods/buurten.shp").drop(columns=["WIJKBUURTC","WIJKCODE","OCE_ID"])

In [44]:
buurten

,BUURTCODE,BUURTNAAM,OPPERVLAKT,OMTREK,geometry
0,108,Bosweide,7.366720e+05,3506.110193,"POLYGON ((4.35666 52.04787, 4.35698 52.04807, ..."
1,109,Tedingerbroek,4.952767e+05,2896.851611,"POLYGON ((4.36732 52.05584, 4.36856 52.05687, ..."
2,110,De Reef,6.209261e+05,3397.598099,"POLYGON ((4.37121 52.05909, 4.37350 52.06104, ..."
3,104,Lage Veld,4.612014e+05,3052.161629,"POLYGON ((4.28727 52.02384, 4.28750 52.02396, ..."
4,105,Zonne Veld,4.996797e+05,3298.689238,"POLYGON ((4.29001 52.01862, 4.29059 52.01883, ..."
...,...,...,...,...,...
109,74,Duttendel,1.330448e+06,6453.778583,"POLYGON ((4.29198 52.10843, 4.29222 52.10858, ..."
110,111,De Venen,3.536171e+05,3563.034190,"POLYGON ((4.38442 52.05377, 4.38445 52.05378, ..."
111,112,Morgenweide,8.817128e+05,4398.716143,"POLYGON ((4.36419 52.04370, 4.36430 52.04380, ..."
112,113,Singels,5.714747e+05,3308.296493,"POLYGON ((4.35466 52.03588, 4.35466 52.03596, ..."
